In [41]:
import numpy as np
import scipy.linalg as la

def pretty_print(name, matrix):
  """Pretty prints a matrix with a title.

  Args:
      name: The name of the matrix.
      matrix: The matrix to be printed.
  """

  print(f"\n{name}:\n")
  np.set_printoptions(precision=4, suppress=True)
  print(matrix)

def skew_screw(S_i):
    """Computes the skew-symmetric matrix of a 6x1 screw-axis vector."""

    w1 = S_i[0]
    w2 = S_i[1]
    w3 = S_i[2]
    v1 = S_i[3]
    v2 = S_i[4]
    v3 = S_i[5]
    return np.array([
        [  0, -w3,  w2, v1],
        [ w3,   0, -w1, v2],
        [-w2,  w1,   0, v3],
        [  0,   0,   0,  0],
    ])

def Ad(T):
    """Computes the adjoint (6x6) of a 4x4 homogeneous transformation matrix."""
    R = T[:3, :3]
    p = T[:3, 3]
    Ad_T = np.block([[R, np.zeros((3, 3))],
                     [np.cross(p, R), R]])
    return Ad_T

def sphere_to_wrench(theta, phi, magnitude):
    # Convert polar and azimuthal angles to Cartesian coordinates
    x = magnitude * np.sin(theta) * np.cos(phi)
    y = magnitude * np.sin(theta) * np.sin(phi)
    z = magnitude * np.cos(theta)

    # Create the wrench vector
    F = np.array([0, 0, 0, x, y, z])

    return F

def compute_space_jacobian(S, theta):
    """Computes the space Jacobian.

    Args:
        S (list or np.ndarray): List of screw axes.
        theta (list or np.ndarray): List of joint angles.

    Returns:
        np.ndarray: Space Jacobian.
    """

    n = len(S)
    J_s = np.zeros((6, n))
    Ad_chain = np.eye(6)

    for i in range(1, n + 1):
        if i == 1:
            J_s[:, i - 1] = S[i - 1]
        else:
            S_i_skew = skew_screw(S[i - 1])  # Extract rotational component and create skew-symmetric matrix
            M_i = la.expm(S_i_skew * theta[i - 1])
            Ad_i = Ad(M_i)
            Ad_chain = Ad_chain @ Ad_i
            J_s[:, i - 1] = Ad_chain @ S[i - 1]

    return J_s

def compute_transformation_matrix(S, theta, M_home):
    """Computes the transformation matrix T_sb.

    Args:
        S (list or np.ndarray): List of screw axes.
        theta (list or np.ndarray): List of joint angles.
        M_home (np.ndarray): End-effector configuration at home position.

    Returns:
        np.ndarray: Transformation matrix T_sb.
    """

    M_list = []
    for S_i, theta_i in zip(S, theta):
        M = la.expm(skew_screw(S_i) * theta_i)
        M_list.append(M)

    T_sb = np.linalg.multi_dot(M_list + [M_home])
    return T_sb

# Link lengths
L1 = 0.026
L2 = 0.026

# Screw axes as 6x1 NumPy arrays
S1 = np.array([1, 0, 0, 0, 0, 0])
S2 = np.array([0, 1, 0, 0, 0, 0])
S = [S1, S2]

# Joint angles
theta1 = np.pi/2
theta2 = 0
theta = [theta1, theta2]

# End-effector configuration at the home position
M_home = np.array([
    [ 0, -1, 0,     0],
    [ 0,  0, 1, L1+L2],
    [-1,  0, 0,     0],
    [ 0,  0, 0,     1],
])

# Wrench applied to the end-effector in the body frame (6x1)
F_b = -1*sphere_to_wrench(np.pi/2, 0, 75)

# Compute Transformation Matrix T_sb
T_sb = compute_transformation_matrix(S, theta, M_home)

# Compute Inverse Transformation Matrix T_bs
T_bs = np.linalg.inv(T_sb)

# Compute Adjoint of Transformation Matrix T_bs
Ad_T_bs = Ad(T_bs)

# Compute space Jacobian J_s
J_s = compute_space_jacobian(S, theta)

# Compute body Jacobian J_b
J_b = Ad_T_bs @ J_s

# Compute Joint Torques τ
tau = J_b.T @ F_b

pretty_print("Joint Configurations (theta)", theta)
pretty_print("Screw Axes (S)", S)
pretty_print("Body Wrench (F_b)", F_b)
pretty_print("Space Jacobian (J_s)", J_s)
pretty_print("Body Jacobian (J_b)", J_b)
pretty_print("Transformation Matrix (T_sb)", T_sb)
pretty_print("Transformation Matrix (T_bs)", T_bs)
pretty_print("Joint Torques (τ)", tau)


Joint Configurations (theta):

[1.5707963267948966, 0]

Screw Axes (S):

[array([1, 0, 0, 0, 0, 0]), array([0, 1, 0, 0, 0, 0])]

Body Wrench (F_b):

[ -0.  -0.  -0. -75.  -0.  -0.]

Space Jacobian (J_s):

[[1. 0.]
 [0. 1.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]

Body Jacobian (J_b):

[[ 0.     1.   ]
 [-1.     0.   ]
 [ 0.     0.   ]
 [ 0.052  0.   ]
 [ 0.     0.052]
 [ 0.     0.   ]]

Transformation Matrix (T_sb):

[[ 0.    -1.     0.     0.   ]
 [ 1.     0.     0.     0.   ]
 [ 0.     0.     1.     0.052]
 [ 0.     0.     0.     1.   ]]

Transformation Matrix (T_bs):

[[ 0.     1.     0.     0.   ]
 [-1.    -0.    -0.    -0.   ]
 [ 0.     0.     1.    -0.052]
 [ 0.     0.     0.     1.   ]]

Joint Torques (τ):

[-3.9  0. ]
